In [20]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
import os
import cc3d
from scipy.io import loadmat, savemat
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
import sknw
import networkx as nx
import pandas as pd
from scipy.ndimage import distance_transform_edt

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

In [ ]:
directory_seg = Path('james_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [ ]:
#for i in tqdm(range(len(files_seg))):
#    tmp = np.load(files_seg[i])
#    tmp = np.reshape(tmp,(1,507,507,252))
#    io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

In [ ]:
files_seg  = list(directory_seg.glob('*_seg.tif'))
files_seg = sorted([x.as_posix() for x in files_seg])
files_seg[0]

In [ ]:
image_path = Path('../TBI')
images = list(image_path.glob('vbm*/*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

In [ ]:
for i in tqdm(range(len(images))):
    image = io.imread(images[i])
    image = np.swapaxes(image,0,1)
    image = np.swapaxes(image,1,3)
    image = resize(image,(2,507,507,252),preserve_range=True)
    zeros = np.zeros((1,507,507,252)).astype('uint16')
    image = np.append(image,zeros,axis = 0)
    image = image.astype('float16')
    io.imsave('james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

In [ ]:
#'james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name

In [5]:
pairs = {"009":"011",
         "010":"012",
         "015":"017",
         "016":"018",
         "021":"023",
         "022":"024",
         "027":"029",
         "028":"030",
         "033":"035",
         "034":"036",
         "039":"041",
         "040":"042",
         "046":"048",
         "047":"049"}
keys = list(pairs.keys())
images = ['james_preds/XYZres'+x+'_upsamp.tif' for x in keys]
targets = ['james_preds/XYZres'+pairs[x]+'_upsamp.tif' for x in keys]
print(targets)

['james_preds/XYZres011_upsamp.tif', 'james_preds/XYZres012_upsamp.tif', 'james_preds/XYZres017_upsamp.tif', 'james_preds/XYZres018_upsamp.tif', 'james_preds/XYZres023_upsamp.tif', 'james_preds/XYZres024_upsamp.tif', 'james_preds/XYZres029_upsamp.tif', 'james_preds/XYZres030_upsamp.tif', 'james_preds/XYZres035_upsamp.tif', 'james_preds/XYZres036_upsamp.tif', 'james_preds/XYZres041_upsamp.tif', 'james_preds/XYZres042_upsamp.tif', 'james_preds/XYZres048_upsamp.tif', 'james_preds/XYZres049_upsamp.tif']


In [9]:
for i in tqdm(range(len(images))):
    print("next")
    fix = io.imread(images[i])
    fix = ants.from_numpy(fix[0].astype('float32'),
                         origin=(0,0,0),
                         spacing=(1.0,1.0,1.0),
                         direction = np.array([[1.0, 0.0, 0.0],
                                               [0.0, 1.0, 0.0],
                                               [0.0, 0.0, 1.0]]))
    mov = io.imread(targets[i])
    mov = ants.from_numpy(mov[0][:,:,::-1].astype('float32'),
                         origin=(0,0,0),
                         spacing=(1.0,1.0,1.0),
                         direction = np.array([[1.0, 0.0, 0.0],
                                               [0.0, 1.0, 0.0],
                                               [0.0, 0.0, 1.0]]))
    fix_mask = io.imread(re.sub('upsamp','seg',images[i]))
    fix_mask = ants.from_numpy(fix_mask.astype('float32'),
                         origin=(0,0,0),
                         spacing=(1.0,1.0,1.0),
                         direction = np.array([[1.0, 0.0, 0.0],
                                               [0.0, 1.0, 0.0],
                                               [0.0, 0.0, 1.0]]))
    mov_mask = io.imread(re.sub('upsamp','seg',targets[i]))
    mov_mask = ants.from_numpy(mov_mask[:,:,::-1].astype('float32'),
                         origin=(0,0,0),
                         spacing=(1.0,1.0,1.0),
                         direction = np.array([[1.0, 0.0, 0.0],
                                               [0.0, 1.0, 0.0],
                                               [0.0, 0.0, 1.0]]))
    #fix = ants.image_read(images[i])
    #mov = ants.image_read(targets[i])
    #fix_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
    #mov_mask = ants.image_read(re.sub('upsamp','seg',targets[i]))
    mytx = ants.registration(fixed = fix,
                            moving = mov,
                            type_of_transform = 'Rigid'
                            )
    warpedmask = ants.apply_transforms(fixed = fix_mask,
                                       moving = mov_mask,
                                       transformlist = mytx['fwdtransforms'],
                                       interpolator = 'nearestNeighbor'
                                      )
    io.imsave(re.sub('seg','seg_warped_mov_baseline',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
    io.imsave(re.sub('seg','seg_warped_fix_baseline',re.sub('upsamp','seg',images[i])),fix_mask.numpy())

  0%|                                                    | 0/14 [00:00<?, ?it/s]

next


  7%|███▏                                        | 1/14 [00:33<07:18, 33.74s/it]

next


 14%|██████▎                                     | 2/14 [01:05<06:33, 32.75s/it]

...

 86%|████████████████████████████████████▊      | 12/14 [06:38<01:06, 33.13s/it]

next


 93%|███████████████████████████████████████▉   | 13/14 [07:12<00:33, 33.46s/it]

next


100%|███████████████████████████████████████████| 14/14 [08:18<00:00, 35.62s/it]


In [10]:
re.sub('seg','seg_warped_mov_baseline',re.sub('upsamp','seg',images[i]))

'james_preds/XYZres047_seg_warped_mov_baseline.tif'

## Save Matlab .mat file of registered images

In [29]:
directory_seg = Path('james_preds')
images = list(directory_seg.glob('*_fix_baseline.tif'))
images = sorted([x.as_posix() for x in images])[::-1]

In [ ]:
for image in tqdm(images):
    #if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_fix','_mov',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_fix_baseline.tif','_seg_warped_single_baseline.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

 43%|██████████████████▊                         | 6/14 [01:27<01:56, 14.56s/it]

In [15]:
re.sub('_fix_baseline.tif','_seg_warped_single_baseline.mat',re.sub('_0001','',image))

'james_preds/XYZres009_seg_warped_seg_warped_single_baseline.mat'

# Gen Graphs

In [7]:
directory = Path('james_preds')
files_seg_0001 = directory.glob('*_seg_warped_mov_baseline.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001)

14

In [10]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_seg_warped_mov_baseline.tif','_seg_warped_skel_warped_single_baseline.mat',file)
    #seg = io.imread(file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    io.imsave(re.sub('_seg_warped_mov_baseline.tif','_single_skel_baseline.tif',file),skel)
    #dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.00001
    #skel_dst = skel*dst_tsfm
#
    #
    #np.save(re.sub('seg','dst_skel',file),skel_dst)
    #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel, multi=False)
    print(len(graph.edges))
    #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    
    nx.write_gpickle(graph,re.sub('_seg_warped_mov_baseline.tif','_warped_baseline.pickle',file))

  0%|                                                    | 0/14 [00:00<?, ?it/s]/tmp/ipykernel_3476067/1851155028.py:7: UserWarning:

james_preds/XYZres009_single_skel_baseline.tif is a low contrast image

  7%|███▏                                        | 1/14 [00:00<00:11,  1.17it/s]

401


/tmp/ipykernel_3476067/1851155028.py:7: UserWarning:

james_preds/XYZres010_single_skel_baseline.tif is a low contrast image

 14%|██████▎                                     | 2/14 [00:01<00:10,  1.15it/s]

358


/tmp/ipykernel_3476067/1851155028.py:7: UserWarning:

james_preds/XYZres015_single_skel_baseline.tif is a low contrast image

 21%|█████████▍                                  | 3/14 [00:02<00:09,  1.17it/s]

...

337


/tmp/ipykernel_3476067/1851155028.py:7: UserWarning:

james_preds/XYZres046_single_skel_baseline.tif is a low contrast image

 93%|███████████████████████████████████████▉   | 13/14 [00:11<00:00,  1.12it/s]

633


/tmp/ipykernel_3476067/1851155028.py:7: UserWarning:

james_preds/XYZres047_single_skel_baseline.tif is a low contrast image

100%|███████████████████████████████████████████| 14/14 [00:12<00:00,  1.14it/s]

22


In [18]:
directory = Path('james_preds')
files = directory.glob('*_warped_baseline.pickle')
files = sorted([x.as_posix() for x in files])
len(files)

14

In [24]:
i=0
xls = pd.ExcelFile('p3_metalog.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(files[::-1]):
    graph = nx.read_gpickle(file)
    if len(graph.edges) < 1000:
        seg_file = re.sub('_warped_baseline.pickle','_seg_warped_fix_baseline.tif',file)
        seg_0001_file = re.sub('_warped_baseline.pickle','_seg_warped_mov_baseline.tif',file)
        seg = io.imread(seg_file)
        seg_0001 = io.imread(seg_0001_file)
        seg_dst = distance_transform_edt(seg)
        seg_0001_dst = distance_transform_edt(seg_0001)
        for sheet_name in xls.sheet_names:
            if re.sub('james_preds/','',re.sub('_warped_baseline.pickle','',file)) in df[sheet_name].values:
                subj = sheet_name
                #if subj in ["TBI07_3D","TBI11_3D","TBI22_3D","TBI31_3D","TBI38_3D","SHAM09_3D","SHAM12_3D","SHAM23_3D","SHAM32_3D"]:
                #    gender = 'male'
                #else:
                #    gender = 'female'
                #treatment = re.sub('SHA','SHAM',subj[0:3])
                _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('james_preds/','',re.sub('_warped_baseline.pickle','',file))]
                if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                    wavelength = _tmp['Unnamed: 11'].iloc[0]
                    power_per = _tmp['Unnamed: 10'].iloc[0]
                    start_depth = _tmp['Unnamed: 2'].iloc[0]
                    for i in range(len(graph.edges)):
                        path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                        _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                    nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))
                    i+=1
print(i)

  7%|███▏                                        | 1/14 [00:19<04:13, 19.48s/it]/tmp/ipykernel_3476067/2933112158.py:50: RuntimeWarning:

invalid value encountered in sqrt

100%|███████████████████████████████████████████| 14/14 [05:42<00:00, 24.43s/it]

401


In [25]:
directory = Path('james_preds_graphs')
files = directory.glob('*_warped_baseline_radii.pickle')
files = sorted([x.as_posix() for x in files])
len(files)
files[0]

'james_preds_graphs/XYZres009_warped_baseline_radii.pickle'

In [26]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))

100%|███████████████████████████████████████████| 14/14 [00:10<00:00,  1.40it/s]
